## Data Analysis SQL Project

> Data manipulaion and Data Execution



- Undesrtand the business problem
- Importing liabraries
- Database connection to extract data
- Exploring the tables present in the Database to identify the Key Variables
- Analyzing the key variables
- Creating Reportwith all the results and Analysis for company

### bjective
* Airline SQL database project data analysis to improve it occupancy (profit margin) and reduced toperational expenses


### tImporting Libraries

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot
import seaboarn as sns

import warnings
warnings.filterwarnings("ignore")



#### Database Connection

In [ ]:
# make connection with travel SQL database
connection = sqlite3.connect('trave.sqlite')
cursor = connection.cursor()


#### Data Exploration

In [ ]:
# Once connection established, we need to explore the database
cursor.execute("""select name from sqlite_master where type = 'table';""")
print('List of table present in database')

# To fetch and list all the cursor tables
table_list = [table[0] for table in cursor.fetchall()]
table_list  # Output: List of tables in database


# To find rows in the executed cursor, We also this method
for row in cursor:
    print(row)


In [ ]:
#    We now can epxplore the tables data
# To data from SQL table the method is read_sql_query
# we will pass the Select query and Connection paramemeter
aircrafts_data = pd.read_sql_query("select * from aircrafts_data", connection)
aircrafts_data.head()
## Or
aircrafts_data

### We also can find the number of Rows and Columns, with 
aircrafts_data.shape    # Output: (9, 3) 9- rows, 3-columns


In [ ]:
# Now read another table, named airport_data
airports_data = pd.read_sql_query("select * from airports_data", connection)
airports_data

In [ ]:
# Third table is boarding_passes
boarding_passes = pd.read_sql_query("select * from boarding_passes", connection)
boarding_passes

In [ ]:
# Forth table is bookings
bookings = pd.read_sql_query("select * from bookings", connection)
bookings

In [ ]:
##### Similarly open and explore all the database tables data, 
# like, seats, tickets, flights, 

# For example, let's say we want to see what's in our 'users' table:
table_data = pd.read_sql_query("SELECT * FROM users;", connection)
print(table_data)


In [ ]:
for table in table_list:
    column_info = connection.execute("PRAGMA table_info({})".format(table))
    print("\nTable: {}".format(column_info)) # if this fetch the column data then comment this line
    # Or
    print('\ntable:', table)

    
# for table in table_list:
#     column_info = connection.execute("PRAGMA table_info({})".format(table))
#     print("\nTable: {}".format(table))
#     columns = [column[1] for column in column_info.fetchall() if not column[2]]  # ignore primary key
#     print(", ".join(columns))


    # Now fetch all the columns
    for column in column_info.fetchall():
        # print("Column Name: {}, Data Type: {}, Not Null? {}")
        print(column[1:3])  # we also slice the indexing [1:3]

    
    

> Check for missing values in tables


In [ ]:
# Now check for the missing values data in database
for table in table_list:
    print('\ntable:', table)
    # call dataframe df variable and sstore all the tables data
    df_table = pd.read_sql_query(f"select * from {table}", connection)
    print(df_table.isnull().sum()) 
    # If value against each row shows 0, its means no missing data found, and the data is complete, consistent and clean.




### Basic Analysis

#### How many planes have more than 100 seats?

In [ ]:
# Count the total seats for each aircraft_code, and find if seats > 100
pd.read_sql_query("""select aircraft_code, count(*) as num_seats
                  group by aircraft_code having num_seats > 100""", connection)
                # we used 'Group by' and 'having' clause in query 


##### How much number of tickets booked and Total amount earned cheanged with time

> First we need to check Number of Tickets per day

In [ ]:
# For calulation Time, we need a line chart 
# Need Join to connect two tables
tickets = pd.read_sql_query("""select * from tickets inner join bookings
                  on tickets.book_ref = bookings.book_ref""", connection)
                # we will recieve the Total amount

# To extract the date/time for the date column in tickets
tickets['book_date'] = pd.to_datetime(tickets('book_date'))
# tickets 

## If we need to remove the time and only needed the date, then
tickets['book_date'] = tickets['book_date'].dt.date()
tickets




In [ ]:
### Now need to group by date column, to find the  total number of tickets for each day
x = tickets.groupby('date')[['date']].count()

plt.figure(figsize = (16,6))

plt.plot(x.index, x['date'], marker = '^')

plt.xlabel('Date', fontsize = 20)
plt.ylabel('Number of Tickets', fontsize = 20)

plt.grid('b')
plt.show()

# sns.heatmap(x)



> Now its the time to find the Total amount earned each day from bookings

In [ ]:
bookings = pd.read_sql_query("""select * from bookings""", connection)
                # we will recieve all bookings

# To extract the date/time for the date column in bookings table
bookings['book_date'] = pd.to_datetime(bookings('book_date'))

## If we need to remove the time and only needed the date, then
bookings['book_date'] = bookings['book_date'].dt.date()

### Now need to group by date column, to find the total amount with sum()
x = bookings.groupby('date')[['total_amount']].sum()

# Draw a plot
plt.figure(figsize = (16,6))
plt.plot(x.index, x['total_amount'], marker = '^')

plt.xlabel('Date', fontsize = 20)
plt.ylabel('Total Amount Earned', fontsize = 20)

plt.grid('b')
plt.show()

##### Calculate the average charges for each aircraft with different fare conditions?

In [ ]:
df = pd.read_sql_query("""select fare_condition s, aircrafts_code, avg(amount)
                       from tickets_flights join flights on ticket_flights.flight.id = flights.flight.id
                       group by aircraft_code, fare_conditions""", connection)

# if we print the df value it will provide the both the tables in joint form.
# Now we want to make a bar chart  for aircraft_code and avg amount, based on fare_conditions.
sns.barplot(data = df, x = 'aircraft_code', y = 'avg(amount)', hue = 'fare_conditions')

##### Example: Create New User, into Users table

In [ ]:
###### EXAMPLE: create table for users
cursor.execute("""CREATE TABLE IF NOT EXISTS Users (
               id INTEGER PRIMARY KEY,
               username TEXT NOT NULL UNIQUE,
               password TEXT NOT NULL)""")
# commit the changes and close the connection to the database
connection.commit()
connection.close()